In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install nilearn

     |████████████████████████████████| 10.0 MB 4.5 MB/s 


In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 3.4 MB/s 


In [ ]:
# importing all the required libraries
import pandas as pd
import numpy as np

# For NiLearn
from nilearn import datasets
from nilearn import plotting
from nilearn import image
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
import numpy as np
from keras.models import load_model

/usr/local/lib/python3.7/dist-packages/nilearn/datasets/__init__.py:96: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [ ]:
datasets.fetch_abide_pcp(data_dir="/content/sample_data", n_subjects=1, pipeline='cpac')


Dataset created in /content/sample_data/ABIDE_pcp



 ...done. (2 seconds, 0 min)
/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:2349: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


Downloaded 102481920 of 104419884 bytes (98.1%,    0.2s remaining) ...done. (10 seconds, 0 min)


{'description': b'ABIDE\n\n\nNotes\n-----\nThe Autism Brain Imaging Data Exchange (ABIDE) dataset provides previously\ncollected resting state functional magnetic resonance imaging datasets\nfrom 539 individuals with ASD and 573 typical controls for the purpose\nof data sharing in the broader scientific community. This grass-root\ninitiative involved 16 international sites, sharing 20 samples yielding\n1112 datasets composed of both MRI data and an extensive array of\nphenotypic information common across nearly all sites (see below).\n\nNote that this is the preprocessed version of ABIDE provided by the\npreprocess connectome projects (PCP).\n\n\nContent\n-------\n    :\'phenotypic\': Behavioral information.\n\n\nReferences\n----------\n\nFor more information about this dataset\'s structure:\nhttp://preprocessed-connectomes-project.github.io\nhttp://www.childmind.org/en/healthy-brain-network/abide/\n\nNielsen, Jared A., et al. "Multisite functional connectivity MRI\nclassification of a

In [ ]:
  # Parameters for the sample size and study
  n_subjects = 40#1112
  DX_GROUP = {1,2}
  DSM_IV_TR = [0,1,2,3,4]
  #data_dir = '/content/drive/Shareddrives/Final Project/Colabs/Data/ABIDE_NILEARN'
  data_dir='/content/drive/Shareddrives/Final Project/FmriTrainTestData'
  #PRE_PROCESSED_FEATURE_DATA = '/content/drive/Shareddrives/Final Project/Colabs/Data/ABIDE_NILEARN/ABIDE_PREPROCESSED_DATA_FEATURES_EXTRACTED'
  PRE_PROCESSED_FEATURE_DATA ='/content/drive/Shareddrives/Final Project/FmriTrainTestData/ABIDE_FEATURE_SET'
  LABEL_DATA='/content/drive/Shareddrives/Final Project/FmriTrainTestData/ABIDE_LABELS'
  MODEL_STORAGE_PATH = '/content/drive/Shareddrives/Final Project/Colabs/Machine Learning Deep Learning/Models/Pre_Processed_ABIDE_Model'

In [ ]:
#apply Smith’s rs-fMRI components atlas. 
#Smith atlas reflects seventy resting-state networks (RSN) acquired using an independent component analysis (ICA) on thousands of healthy patients.
smith_atlas = datasets.fetch_atlas_smith_2009()
smith_atlas_rs_networks = smith_atlas.rsn70


Dataset created in /root/nilearn_data/smith_2009



Downloaded 11132928 of 15129514 bytes (73.6%,    1.1s remaining) ...done. (5 seconds, 0 min)


 ...done. (1 seconds, 0 min)


Downloaded 51191808 of 53028809 bytes (96.5%,    0.2s remaining) ...done. (6 seconds, 0 min)


Downloaded 15163392 of 19114114 bytes (79.3%,    0.5s remaining) ...done. (4 seconds, 0 min)


Downloaded 8019968 of 9566632 bytes (83.8%,    0.2s remaining) ...done. (2 seconds, 0 min)


Downloaded 65200128 of 67017538 bytes (97.3%,    0.2s remaining) ...done. (8 seconds, 0 min)


In [ ]:
# Region signals extraction using a mask
# exctracting the activation values of the seventy networks
from nilearn.input_data import NiftiMapsMasker
# Generate a mask
masker = NiftiMapsMasker(maps_img=smith_atlas_rs_networks,  # Smith atlas
                         standardize=True, # centers and norms the time-series
                         memory='nilearn_cache', # cache
                         verbose=0) #do not print verbose
            
# Apply the mask on the data & sort it based on label

## NOTE: **func_file** below is **nii.gz** file formatted fMRI file

In [ ]:
func_file = "/content/sample_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051203_func_preproc.nii.gz"

In [ ]:
subject_data = masker.fit_transform(func_file)

/usr/local/lib/python3.7/dist-packages/nilearn/image/image.py:1109: FutureWarning: The parameter "sessions" will be removed in 0.9.0 release of Nilearn. Please use the parameter "runs" instead.
  ensure_finite=ensure_finite)


In [ ]:
import numpy as np

# get longest scan
max_len_image=np.max(len(subject_data))

# reshape data

  # Padding
N= max_len_image-len(subject_data)
padded_array=np.pad(subject_data, ((0, N), (0,0)), 
                    'constant', constant_values=(0))
subject_data=padded_array
subject_data=np.array(subject_data)
subject_data = subject_data.reshape(1,subject_data.shape[0],subject_data.shape[1])
subject_data = np.array(subject_data)
  
print ('data shape: ',subject_data.shape)

data shape:  (1, 196, 70)


## With model

> Indented block



In [ ]:
loaded_model = load_model("/content/drive/Shareddrives/Final Project/Colabs/Machine Learning Deep Learning/Models/Pre_Processed_ABIDE_Model/bi_lstm_model/")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
print('evaluating..')
y_pred=loaded_model.predict(subject_data)
y_pred_1d=1.0 if y_pred[0][0]>.5 else 0.0 
y_pred_1d

evaluating..


1.0

## Endpoint

### Two parts to integrating the endpoint:
#### Save the file to google cloud stoarge. Use the json(nice-abbey-key.json) in the folder(/content/drive/Shareddrives/Final Project/) to gain access. Then push the file to the storage bin as shown below. 
#### Call the endpoint to get the result. It is an array - you can either average or take the first value to determine classification. 

In [ ]:
!pip install google-cloud-storage


In [ ]:
from google.cloud import storage
client = storage.Client.from_service_account_json(json_credentials_path='/content/drive/Shareddrives/Final Project/nice-abbey-key.json')
bucket = client.get_bucket('fmri_data_musketeers')
object_name_in_gcs_bucket = bucket.blob('Pitt_0050008_func_preproc.nii.gz')
object_name_in_gcs_bucket.upload_from_filename("/content/drive/Shareddrives/Final Project/FmriTrainTestData/ABIDE_pcp/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050008_func_preproc.nii.gz")

In [ ]:
# importing the requests library
import requests
import base64
import json

# defining the api-endpoint 
API_ENDPOINT = "https://bidlstm-o-yjmgrhmjwq-uc.a.run.app/predict"
#API_ENDPOINT = "http://127.0.0.1:32775/predict"
# importing the requests library
nii_file = {'nii_file':'Pitt_0050006_func_preproc.nii.gz'}

r = requests.post(url = API_ENDPOINT, json = nii_file)

In [ ]:
r.text

'{"predictions":[[0.5635077357292175,0.4610428512096405]]}\n'

In [ ]:
import numpy as np
y_preds = json.loads(r.text.replace("\n",""))['predictions'][0]
print(y_preds[0])
conval = np.mean(y_preds[0]) > 0.5 
print(conval)
if y_preds[0] >.57 :
  diag_class = 1.0 
else :
  diag_class = 0.0
print(diag_class) 

0.5635077357292175
True
0.0


In [ ]:
from google.cloud import storage
client = storage.Client.from_service_account_json(json_credentials_path='/content/drive/Shareddrives/Final Project/nice-abbey-key.json')
bucket = client.get_bucket('fmri_data_musketeers')

In [63]:
# importing the requests library
import requests
import base64
import json

In [64]:
# defining the api-endpoint 
API_ENDPOINT = "https://bidlstm-o-yjmgrhmjwq-uc.a.run.app/predict"
#API_ENDPOINT = "http://127.0.0.1:32775/predict"
# importing the requests library
nii_file = {'nii_file':'Pitt_0050007_func_preproc.nii.gz'}

r = requests.post(url = API_ENDPOINT, json = nii_file)

In [65]:
r.text

'{"predictions":[[0.5286981463432312,0.46851107478141785]]}\n'

In [ ]:
import numpy as np
y_preds = json.loads(r.text.replace("\n",""))['predictions'][0]
conval = np.mean(y_preds) > 0.5 
diag_class = 1.0 if conval >.5 else 0.0
print(diag_class) 

0.0


In [66]:
y_preds = json.loads(r.text.replace("\n",""))['predictions'][0]
print(y_preds[0])
conval = np.mean(y_preds[0]) > 0.5 
print(conval)
if y_preds[0] >.57 :
  diag_class = 1.0 
else :
  diag_class = 0.0
print(diag_class) 

0.5286981463432312
True
0.0
